# Functional Programming in Python
## Introduction
Functional programming is an approach to programming where programs are built by composing and running functions that perform a series of transformation on data. This is in contrast with the common approach of *imperative programming*, where programs are written as series of statements which modify the *state* of the computation environment. Normally within functional programming, great focus is placed on *composability*, *immutability* and  *purity*. We are going to define these terms in more detail later.
## Why Functional Programming
Why do we choose functional programming? There are a series of advantages to this approach, namely:
- **Debugging** and **testing** are easy: there are no surprises because every function only does one thing and does not affect any other piece of the program.
- **Parallelisation** is trivial: because functions are just small boxes that take one input and produce an output and do not depend implicitely on other parts of the code, it is easy to make several functions run in parallel.
## The basic principles of functional programming
All modern programming languages have *functions* (or methods, procedures, subroutines, subprograms); these are groups of program statements that perform a certain computation. Functions are defined once for the whole program and can be reused at will throughout the program whenever we need to perform the specific computation they are defined for. Using functions, we can split our code in smaller units that are only responsible for a specific *functionality*; this helps us structuring our code in a clean and understandable form. 
### Pure Functions
In functional programming, we try to strive for *purity*, that is we want to define and use functions that only depend on their input, always return the same output for the same inputs and do not have any *side effects*, that is they do not indirectly affect any other part of our program. You can think of these functions as mathematical functions. 
Other examples of side effects are:
- printing to the program output
- reading or writing files
- generating and using random numbers
To besser understand this concept, let us look at the function `my_first_pure_function` we defined below:


In [1]:
def my_first_pure_function(x: int) -> int: 
    return x + 1

&#x1F600
Is this a pure function?
As you could probably imagine, this function is pure. Any time we run it, we get the same  result and running the function does not affect any other part of our program:

In [2]:
x = 1
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")

x is 1
The result of calling the function is 2
x is 1
The result of calling the function is 2
x is 1


As a rule of thumb, any function that does not modify variables outside of its *scope* and only uses mathematical functions is a pure function.
Now, consider this function instead:

In [3]:
x = ["short", "list"] 
def do_something(y: str) -> None:
    x.append(y)

> &#x1F600 Is this function pure? 

Let's try and run it. 

In [4]:
print(x)
do_something("a")
print(x)

['short', 'list']
['short', 'list', 'a']


As you see from the output, the function modified the list `x`. Therefore, this function is not pure. This leads us to the next principle, **immutability**.

### Immutability
When writing programs in functional style, we usually avoid functions like `do_something`. Instead of modifying existing data (*mutation*), you write functions that transform your data and return new objects.
In the case of the function above, we would rewrite it as follows:


In [5]:
x = ["short", "list"] 
def do_something_immutable(x: list[str], y: str) -> list[str]:
    return x + [y]

print(x)
print(do_something_immutable(x, "a"))
print(x)

['short', 'list']
['short', 'list', 'a']
['short', 'list']


The output shows that `do_something_immutable` does not change `x`. It also does not reference to `x` outside of the scope but takes it as an argument. Instead of modfying the original `x` list, it returns a new list. When we want to keep immutability, this is the style we work with.

### Higher Order Functions / Functions as Values
Another important principle of functional programming is that **functions are values**. In programming languages (like python) that support a functional style, we can manipulate functions with the language, we can pass them around in a variable and even use functions as parameters for another function.

As an example, consider the function `function_caller`:



In [11]:
from typing import Callable, Any
def function_caller(f: Callable[[Any], Any], arg: Any) -> Any:
    print(f"Calling the function {f} with argument {arg}")
    res = f(arg)
    print(f"The result of the call was {res}")
    return res

This function takes a function `f` and its argument `args` as input and return the result of calling the function, while additionally printing a message on the standard output of the program. Let's try this out.
To do so, we define a new function `add_five`:

In [7]:
def add_five(x: int) -> int:
    return x + 5

Now we can call `function_caller` with `add_five` and another number as arguments:

In [13]:
function_caller(add_five, 4)

Calling the function <function add_five at 0x7fb68848f1c0> with argument 4
The result of the call was 9


9

This example was a bit convoluted, but it shows that in python we can use functions as values and even pass them as arguments to other functions.

This is actually useful in many cases; a typical example being numerical optimisation, where we want to find the parameters to a function that minimise a certain criteria.

Other than these specific use cases, there are some common *higher order functions* (functions that take functions as parameters) that are common in most programming languages. Let's look at some of these in the next paragraph:

#### Mapping
Python offers the `map` function which can be used to apply a given function to all members of an `iterable`. Do not worry too much about what an `iterable` exactly is, just see it as a generalisation of a list, representing all objects that we can `iterate` over, accessing their values one by one. Let's see an example of `map` using the `add_five` function:

In [14]:
map(add_five, [1,2,3,4])

The output is a bit confusing! The reason is that `map` does not return a list but a [map](https://docs.python.org/3/library/functions.html#map) object. This object is `lazy`, that means that the values are only generated when we access them, for example by iterating over the `map` object. 
Fortunately, we can easily convert this into a list by calling the `list` function:

In [15]:
list(map(add_five, [1,2,3,4]))

[6, 7, 8, 9]

This is equivalen to using a for loop:


In [16]:
a = []
for i in [1,2,3,4]:
    a.append(add_five(i))
print(a)

[6, 7, 8, 9]


However, as you can see this former style is not very functional because we mutate an existing list `a` by appending the results of calling `add_five` to it. We will see later that python offers a syntax called *list compehrension* to express this operation in a more functional style.
`map` is very useful when we want to apply the same functions to a list of parameters and we do not want to write a `for` loop. It makes for more coincise code, while sacrificing some readability. 

#### Filtering
Another basic higher order function is `filter`



# Exercises


For each of the f